In [1]:
import numpy as np
import dezero.functions as F

from dezero.core import Variable
from dezero.utils import numerical_diff

### **1. 변수 (Variable)**

In [2]:
data = np.array(1.0)
x = Variable(data)      # x는 Variable의 인스턴스

print(f"val: {x.data}, dim: {x.data.ndim}")

val: 1.0, dim: 0


### **2. 함수 (Function)**

In [3]:
x = Variable(np.array(10))
f = F.Square()    # Function 클래스를 상속
y = f(x)        # 입력으로 사용되는 x는 Variable 인스턴스

print(f"val: {y.data}, type: {type(y)}")

val: 100, type: <class 'dezero.core.Variable'>


### 3. **함수 연결 (Composite Function)**

In [4]:
sqr = F.Square()
exp = F.Exp()

# 합성 함수를 통해 "계산 그래프" 생성
x = Variable(np.array(0.5))
h1 = sqr(x)
h2 = exp(h1)
y = sqr(h2)

print(f"val: {y.data: .4f}, type: {type(y)}")

val:  1.6487, type: <class 'dezero.core.Variable'>


### **4. 수치 미분 (Numerical Differential)**

- 중앙 차분: x + h와 x - h의 값을 이용한 미분 (오차 최소화)
- 정확도: 유효 자릿수 누락으로 인한 오차 발생
- 계산 비용: 편미분 수행 시(변수가 여러 개인 경우), 각 변수를 찾아 연산을 수행해주어야 함

In [5]:
def f(x):
    sqr = F.Square()
    exp = F.Exp()

    return sqr(exp(sqr(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x) # 중앙 차분을 이용한 수치 미분
print(dy)

3.2974426293330694


### **5. 역전파 (Backward Pass)**

- 수치 미분과 달리, 자신과 관계된 정보만을 계산
- 연쇄 법칙을 이용하여 미분을 수행하기 때문에 한 번의 역전파로 미분값 계산 가능

In [6]:
# 서로 다른 인스턴스를 생성해야만 forward 시 저장한 정보가 겹치지 않음
sqr1 = F.Square()
exp = F.Exp()
sqr2 = F.Square()

# 순전파 (Forward pass)
x = Variable(np.array(0.5))
h1 = sqr1(x)
h2 = exp(h1)
y = sqr2(h2)

# 역전파 (Backward pass)
y.grad = np.array(1.0)  # y의 gradient를 1로 설정 (자기 자신이므로)
h2.grad = sqr1.backward(y.grad)
h1.grad = exp.backward(h2.grad)
x.grad = sqr2.backward(h1.grad)

print(x.grad) # 미분값

3.297442541400256


### 6. **자동 미분 (Auto Gradient)**

In [7]:
sqr1 = F.Square()
exp = F.Exp()
sqr2 = F.Square()

# 순전파 (Forward pass)
x = Variable(np.array(0.5))
h1 = sqr1(x)
h2 = exp(h1)
y = sqr2(h2)

# 역전파
y.grad = 1.0
y.backward()
print(x.grad)

3.297442541400256


### **7. 편의성 개선**

- 각 함수 및 클래스의 목적에 맞게 파일 분리 (Refactorizing)
- functions의 각 클래스를 함수로 호출할 수 있도록 추가
- Variable: backward 호출을 재귀에서 반복문으로 변경 / ndarray 자료형만 입력으로 허용
- Function: np.ones_like를 활용하여 data와 동일한 자료형으로 gradient 초기화
- as_array: 0차원 ndarray 계산 결과인 스칼라 값을 다시 array로 변경

In [8]:
x = Variable(np.array(0.5))
y = F.square(F.exp(F.square(x)))
y.backward()
print(x.grad)

3.297442541400256


### **8. 테스트 (Test)**

In [9]:
# 특정 테스트 코드 실행
!python3 -m unittest tests/test_square.py

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


In [10]:
# 모든 테스트 코드 실행
!python3 -m unittest discover tests

...
----------------------------------------------------------------------
Ran 3 tests in 0.001s

OK


### **9. 가변 길이 (Variable Length)**

In [18]:
x0, x1 = Variable(np.array(2.0)), Variable(np.array(3.0))

z = F.add(F.square(x0), F.square(x1))
z.backward()

print(f"forward: {z.data}\nbackward: {x0.grad}, {x1.grad}")

forward: 13.0
backward: 4.0, 6.0


### **10. Variable 완성**

- 같은 변수 반복 사용 문제 해결
- 복잡한 계산 그래프 시 각 함수의 우선순위 문제 해결

In [3]:
x = Variable(np.array(2.0))
a = F.square(x)
y = F.add(F.square(a), F.square(a))
y.backward()

print(f"forward: {y.data}\nbackward: {x.grad}")

forward: 32.0
backward: 64.0
